In [1]:
import kquant as kq
from tqdm import tqdm

In [2]:
def set_api_account():
    kq.set_api("KRX2308020", "EQDkUcyI3dK6oIAXqAR8BXOK4bKxHHmH")
    return None

In [3]:
set_api_account()

In [4]:
def get_all_symbols():
    symbols_df = kq.symbol_stock()
    return symbols_df


def get_filter_symbols_df(symbols_df):
    filtered_symbols_df = symbols_df[
        (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
        & (symbols_df["ADMIN_ISSUE"] == 0)
        & (symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"]))
    ]
    return filtered_symbols_df


def get_symbols(symbols_df):
    symbols = sorted(set(symbols_df["SYMBOL"]))
    return symbols


def get_stock_data(symbol):
    stock_df = kq.daily_stock(symbol)
    return stock_df

In [5]:
symbols_df = get_all_symbols()
filtered_symbols_df = get_filter_symbols_df(symbols_df)
filtered_symbols = get_symbols(filtered_symbols_df)

stock_df_list = list()
for symbol in tqdm(filtered_symbols[:30]):
    _stock_df = get_stock_data(symbol)
    stock_df_list.append(_stock_df)

stock_df = pd.concat(stock_df_list, axis=0)

100%|██████████| 30/30 [00:15<00:00,  1.99it/s]


In [6]:
def get_zero_filtered_df(stock_df):
    _stock_df = stock_df.copy()
    columns = ["VOLUME", "OPEN", "HIGH", "LOW", "CLOSE"]
    for column in columns:
        _stock_df = _stock_df[_stock_df[column] != 0]
    return _stock_df

In [7]:
zero_filtered_stock_df = get_zero_filtered_df(stock_df)

In [13]:
def get_most_common_stock_cnt(stock_df):
    stock_mode_cnt = stock_df.groupby("SYMBOL").count()["CLOSE"].mode()[0]
    return stock_mode_cnt

def get_cnt_filtered_stocks(stock_df, stock_mode_cnt):
    stock_cnt_series = stock_df.groupby("SYMBOL").count()["CLOSE"]
    cnt_filtered_series = stock_cnt_series[stock_cnt_series == stock_mode_cnt]
    cnt_filtered_stocks = cnt_filtered_series.index
    return cnt_filtered_stocks

In [14]:
stock_mode_cnt = get_most_common_stock_cnt(zero_filtered_stock_df)

cnt_filtered_stocks = get_cnt_filtered_stocks(zero_filtered_stock_df, stock_mode_cnt)

In [15]:
zero_cnt_filtered_stock_df = zero_filtered_stock_df[
    zero_filtered_stock_df["SYMBOL"].isin(cnt_filtered_stocks)
]

In [16]:
main_stock_df = zero_cnt_filtered_stock_df.copy()

In [17]:
# Utils
# 1. Sort data by date
def sort_dataset_df(stock_df, column):
    sorted_dataset_df = stock_df.sort_values(column)
    return sorted_dataset_df


In [19]:
main_stock_df

,DATE,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,AMOUNT,CHG,CHG_PCT,CHG_TYPE,EX_EVENT,OPEN_NOTADJ,HIGH_NOTADJ,LOW_NOTADJ,CLOSE_NOTADJ,CHG_NOTADJ,CHG_PCT_NOTADJ,CHG_TYPE_NOTADJ,MARKETCAP,SHARES
0,2022-09-06,000020,"9,460","9,600","9,460","9,510","31,758","302,124,320",10,0.1100,2,00,"9,460","9,600","9,460","9,510",10,0.1100,2,"265,628,279,700","27,931,470"
1,2022-09-07,000020,"9,510","9,510","9,270","9,280","93,307","870,361,280",-230,-2.4200,5,00,"9,510","9,510","9,270","9,280",-230,-2.4200,5,"259,204,041,600","27,931,470"
2,2022-09-08,000020,"9,420","9,420","9,280","9,290","50,448","470,184,060",10,0.1100,2,00,"9,420","9,420","9,280","9,290",10,0.1100,2,"259,483,356,300","27,931,470"
3,2022-09-13,000020,"9,340","9,680","9,340","9,610","80,534","770,646,180",320,3.4400,2,00,"9,340","9,680","9,340","9,610",320,3.4400,2,"268,421,426,700","27,931,470"
4,2022-09-14,000020,"9,400","9,500","9,310","9,480","35,028","329,662,620",-130,-1.3500,5,00,"9,400","9,500","9,310","9,480",-130,-1.3500,5,"264,790,335,600","27,931,470"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2023-08-31,000390,"6,260","6,350","6,210","6,340","65,890","414,045,620",80,1.2800,2,00,"6,260","6,350","6,210","6,340",80,1.2800,2,"172,469,993,460","27,203,469"
245,2023-09-01,000390,"6,380","6,390","6,230","6,340","44,864","281,611,730",0,0.0000,3,00,"6,380","6,390","6,230","6,340",0,0.0000,3,"172,469,993,460","27,203,469"
246,2023-09-04,000390,"6,380","6,390","6,230","6,380","39,396","248,663,210",40,0.6300,2,00,"6,380","6,390","6,230","6,380",40,0.6300,2,"173,558,132,220","27,203,469"
247,2023-09-05,000390,"6,380","6,400","6,280","6,350","16,597","104,933,740",-30,-0.4700,5,00,"6,380","6,400","6,280","6,350",-30,-0.4700,5,"172,742,028,150","27,203,469"


In [ ]:
# Model preprocessing
# 1. Append price diff
def append_price_diff(stock_df, open_col, close_col):
    stock_df["price_diff"] = (stock_df[open_col] - stock_df[close_col]) / stock_df[open_col]
    return stock_df


# 2. Get arraylist
def get_array_list(stock_df, column):
    _arraylist = stock_df[column].values
    return _arraylist